Dataload and cleaning- since it is a continous valued dataset-i.e,iris dataset

In [65]:
from sklearn import datasets
import pandas as pd
import math
import numpy as np
from sklearn.tree import DecisionTreeClassifier

In [66]:
iris = datasets.load_iris() 

In [67]:
df = pd.DataFrame(iris.data)
df.columns = ["sl", "sw", 'pl', 'pw'] 

In [68]:
#TO find and assign labels for respective values of iris dataset since it is a continous valued dataset

def label(val, *boundaries):
    if (val < boundaries[0]):
        return 'a'
    elif (val < boundaries[1]):
        return 'b'
    elif (val < boundaries[2]):
        return 'c'
    else:
        return 'd'


def toLabel(df, old_feature_name):
    second = df[old_feature_name].mean()
    minimum = df[old_feature_name].min()
    first = (minimum + second)/2
    maximum = df[old_feature_name].max()
    third = (maximum + second)/2
    return df[old_feature_name].apply(label, args= (first, second, third))

In [69]:
#Convert all columns to labelled data


df['sl_labelled'] = toLabel(df, 'sl')
df['sw_labelled'] = toLabel(df, 'sw')
df['pl_labelled'] = toLabel(df, 'pl')
df['pw_labelled'] = toLabel(df, 'pw')
df.head

<bound method NDFrame.head of       sl   sw   pl   pw sl_labelled sw_labelled pl_labelled pw_labelled
0    5.1  3.5  1.4  0.2           b           c           a           a
1    4.9  3.0  1.4  0.2           a           b           a           a
2    4.7  3.2  1.3  0.2           a           c           a           a
3    4.6  3.1  1.5  0.2           a           c           a           a
4    5.0  3.6  1.4  0.2           a           c           a           a
..   ...  ...  ...  ...         ...         ...         ...         ...
145  6.7  3.0  5.2  2.3           c           b           c           d
146  6.3  2.5  5.0  1.9           c           a           c           d
147  6.5  3.0  5.2  2.0           c           b           c           d
148  6.2  3.4  5.4  2.3           c           c           d           d
149  5.9  3.0  5.1  1.8           c           b           c           c

[150 rows x 8 columns]>

In [70]:
df.drop(['sl', 'sw', 'pl', 'pw'], axis = 1, inplace = True)

In [71]:
set(df['sl_labelled'])

{'a', 'b', 'c', 'd'}

In [72]:
label_flowers = ['Iris-Setosa(0)','Iris-versicolor(1)','Iris-Virginica(2)']

TO print the steps for decision tree-

In [73]:
def operation(x):                     
    return -1.0*x* math.log(x)

In [80]:
def dict_gen(y):               #to find the unique values of y its frequency and store possible values in a dictionary
    dict ={}
    for i in y[0]:
        dict[i]= dict.get(i,0) +1
    return dict

In [91]:
def entropy(y):             # to calculate the entropy 
    ans=0.0
    dict= dict_gen(y)
    tot = y.shape[0]
    for i in dict:
        ans+= operation(dict[i]/tot)
    return ans

In [92]:
def gain_ratio(ent, df, y, f): # to find the gain_ratio 
    poss_values = set(df[f])
    tot = y.shape[0]
    tot_ent = 0.0
    split_ratio=0.0
    for i in poss_values:
        y_f= y[df[f]==i]
        curr_cnt = y_f.shape[0]
        curr_split= operation(curr_cnt/tot)
        curr_ent = (curr_cnt*entropy(y_f))/tot
        tot_ent+= curr_ent
        split_ratio+=curr_split
    info_gain = ent- tot_ent
    return (info_gain/split_ratio)    

In [99]:
def Tree1(df, y, unused_features,lvl):   
    print("Level ", lvl)
    count = set(y[0])
    z=list(count)
    ent = entropy(y)
    if(len(count)==1):                          #base case1 for unique value of y
        print("Count of ",label_flowers[z[0]]," = ", y.shape[0])
        print("Current Entropy  is = 0.0")
        print("Reached leaf Node \n")
        return
    if(len(unused_features)==0):              #case when no features to split remain
        dict =dict_gen(y)
        for k in dict:
            print("Count of ",label_flowers[k]," = ", dict[k])
        print("Current Entropy  is = ",ent)
        print("Reached leaf Node \n")
        return
    
    dict =dict_gen(y)
    for i in dict:
        print("Count of ",label_flowers[i] ," = ", dict[i] )

    best_gain_ratio = 0.0 
    best_feature = ""
    for f in unused_features:                 # to find the feature which gives best gain ratio
        gain_r = gain_ratio(ent,df,y,f)
        if(gain_r >= best_gain_ratio):
            best_gain_ratio = gain_r
            best_feature = f
      
    print("Splitting on feature ", best_feature, " with gain ratio ", best_gain_ratio, " \n")
    best_feat_val = set(df[best_feature])    
    unused_features.remove(best_feature)    
    for i in best_feat_val:                  
        df_new=df[df[best_feature] == i]
        y_new = y[df[best_feature] == i]
        Tree1(df_new, y_new, unused_features,lvl+1)   
                    
    return 
    

In [100]:
y = pd.DataFrame(iris.target)
unused_features = set(df.columns)
X_test = df
Y_test = y


In [101]:
Tree1(df, y, unused_features, 0)

Level  0
Count of  Iris-Setosa(0)  =  50
Count of  Iris-versicolor(1)  =  50
Count of  Iris-Virginica(2)  =  50
Splitting on feature  pw_labelled  with gain ratio  0.6996382036222091  

Level  1
Count of  Iris-Setosa(0)  =  50
Current Entropy  is = 0.0
Reached leaf Node 

Level  1
Count of  Iris-Virginica(2)  =  34
Current Entropy  is = 0.0
Reached leaf Node 

Level  1
Count of  Iris-versicolor(1)  =  10
Current Entropy  is = 0.0
Reached leaf Node 

Level  1
Count of  Iris-versicolor(1)  =  40
Count of  Iris-Virginica(2)  =  16
Splitting on feature  pl_labelled  with gain ratio  0.43340994956210654  

Level  2
Count of  Iris-Virginica(2)  =  8
Current Entropy  is = 0.0
Reached leaf Node 

Level  2
Count of  Iris-versicolor(1)  =  1
Current Entropy  is = 0.0
Reached leaf Node 

Level  2
Count of  Iris-versicolor(1)  =  39
Count of  Iris-Virginica(2)  =  8
Splitting on feature  sl_labelled  with gain ratio  0.1267450377580933  

Level  3
Count of  Iris-Virginica(2)  =  1
Current Entropy 

Combining all the functions into a class:-

In [106]:
class Decision_Tree:
    def __init__(self,level):
        self.level = level               
        self.feature_splitted_upon = ""  
        self.isleaf = False              
        self.curr_feat_val = ""            
        self.counts = {}                 
        self.children = []              
    
    def fit_int(self,X,Y,unused_features): 
        count = set(Y[0])
        z=list(count)
        ent = entropy(Y)
        if(len(count)==1):                    
            self.isleaf = True
            self.counts[z[0]] = Y.shape[0]
            return
        if(len(unused_features)==0):         
            self.isleaf = True
            dict =dict_gen(Y)
            self.counts= dict
            return
    
        dict =dict_gen(Y)
        self.counts = dict                  
        best_gain_ratio = 0.0 
        best_feature = ""
        for f in unused_features:           
            gain_r = gain_ratio(ent,X,Y,f)
            if(gain_r >= best_gain_ratio):
                best_gain_ratio = gain_r
                best_feature = f
        
        self.feature_splitted_upon = best_feature  
        best_feat_val = set(X[best_feature])
        unused_features.remove(best_feature)
        for i in best_feat_val:
            df_new=X[X[best_feature] == i]
            y_new = Y[X[best_feature] == i]
            obj =  Decision_Tree(self.level +1 )          
            obj.fit_int(df_new, y_new, unused_features)   
            obj.curr_feat_val = i
            self.children.append(obj)                     
        return 

    def fit(self,X,Y):                      
        unused_features= set(X.columns)
        self.fit_int(X,Y,unused_features)
        
    def predict_dum(self,X_test):       
        if(self.isleaf == True):        
            answer = ""
            max_ans=0
            for i in self.counts:
                if(self.counts[i]>max_ans):
                    max_ans = self.counts[i]
                    answer = i
            return answer
        feat = self.feature_splitted_upon 
        answer=""
        for i in self.children:                   
            if(X_test[feat]== i.curr_feat_val):   
                answer= i.predict_dum(X_test)     
                break
        return answer                             
    
    def predict(self,X_test):                     
        result = []
        for i in range(X_test.shape[0]):          
            k = X_test.iloc[i]
            curr_ans = self.predict_dum(k)        
            result.append(curr_ans)               
        return result
        
    

In [107]:
curr_obj = Decision_Tree(0)          
curr_obj.fit(X_test,Y_test)          
pred_res = curr_obj.predict(X_test)  
print(pred_res)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1]


In [108]:
#Comparison between actual and predicted results 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
print(confusion_matrix(Y_test, pred_res))
print(classification_report(Y_test,pred_res))


[[50  0  0]
 [ 0 50  0]
 [ 0  7 43]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           1       0.88      1.00      0.93        50
           2       1.00      0.86      0.92        50

    accuracy                           0.95       150
   macro avg       0.96      0.95      0.95       150
weighted avg       0.96      0.95      0.95       150



In [112]:
iris1 = datasets.load_iris() 
df1 = pd.DataFrame(iris1.data)
df1.columns = ["sl", "sw", 'pl', 'pw'] 
def label(val, *boundaries):
    if (val < boundaries[0]):
        return '1.0'
    elif (val < boundaries[1]):
        return '2.0'
    elif (val < boundaries[2]):
        return '3.0'
    else:
        return '4.0'


def toLabel(df1, old_feature_name):
    second = df1[old_feature_name].mean()
    minimum = df1[old_feature_name].min()
    first = (minimum + second)/2
    maximum = df1[old_feature_name].max()
    third = (maximum + second)/2
    return df1[old_feature_name].apply(label, args= (first, second, third))
df1['sl_labelled'] = toLabel(df1, 'sl')
df1['sw_labelled'] = toLabel(df1, 'sw')
df1['pl_labelled'] = toLabel(df1, 'pl')
df1['pw_labelled'] = toLabel(df1, 'pw')
df1.drop(['sl', 'sw', 'pl', 'pw'], axis = 1, inplace = True)
set(df1['sl_labelled'])
label_flowers = ['Iris1-Setosa(0)','Iris1-versicolor(1)','Iris1-Virginica(2)']
y1 = pd.DataFrame(iris1.target)
unused_features = set(df1.columns)
X1_test = df1
Y1_test = y1


from sklearn.tree import DecisionTreeClassifier
clf=DecisionTreeClassifier()
clf.fit(X1_test,Y1_test)
y_test_pred=clf.predict(X1_test)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
print(confusion_matrix(y_test_pred, pred_res))
print(classification_report(y_test_pred,pred_res))


#comparison with inbuilt sklearn decision tree classification algorithm


[[50  0  0]
 [ 0 57  0]
 [ 0  0 43]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           1       1.00      1.00      1.00        57
           2       1.00      1.00      1.00        43

    accuracy                           1.00       150
   macro avg       1.00      1.00      1.00       150
weighted avg       1.00      1.00      1.00       150

